# AST Core Nodes: Classes, Calls, and Annotations

**Part 2 of the Python AST Guide**

## Prerequisites

This notebook builds on Part 1 which covered AST fundamentals, the NodeVisitor pattern, and function-related nodes. If you haven't completed Part 1, please start there first.

## Overview

This part focuses on the core node types essential for understanding class structures, method calls, and type annotations in Python static analysis:

- **Class Nodes**: Understanding class definitions and nested classes
- **Call Nodes**: Critical for method call analysis and resolution
- **Assignment Nodes**: Key for variable type tracking
- **Name/Attribute Nodes**: Building blocks of variable and method references
- **Type Annotations**: Simple, string, and complex annotation patterns

These concepts are directly applicable to implementing scope-aware variable tracking for static analysis tools and code analysis applications.

In [1]:
# Standard imports for AST analysis
import ast
from typing import Optional, Union, List, Dict, Callable, TypeVar

## 5. Class Nodes

Classes are containers for methods in static analysis. When we find a method inside a class, we need to build its qualified name (like `Calculator.add`) by combining the class name with the method name. Classes can be nested, which is why we maintain a class stack in our visitor to track the current class context.

**ast.ClassDef**

A ClassDef node represents a class definition. The `bases` list contains the base classes (for inheritance), `body` contains all the class contents (methods, class variables, nested classes), and `decorator_list` contains any decorators. For static analysis tools, we primarily care about traversing into the body to find methods and building proper qualified names.

In [2]:
code = """
@dataclass
class Calculator(BaseCalculator, Protocol):
    '''A calculator class'''

    class_var = 10  # Class variable

    def __init__(self):
        self.instance_var = 20  # Instance variable

    def add(self, a: int, b: int) -> int:
        return a + b

    @staticmethod
    def static_method():
        pass

    class NestedClass:
        def nested_method(self):
            pass
"""

tree = ast.parse(code)
print(ast.dump(tree, indent=2))

Module(
  body=[
    ClassDef(
      name='Calculator',
      bases=[
        Name(id='BaseCalculator', ctx=Load()),
        Name(id='Protocol', ctx=Load())],
      body=[
        Expr(
          value=Constant(value='A calculator class')),
        Assign(
          targets=[
            Name(id='class_var', ctx=Store())],
          value=Constant(value=10)),
        FunctionDef(
          name='__init__',
          args=arguments(
            args=[
              arg(arg='self')]),
          body=[
            Assign(
              targets=[
                Attribute(
                  value=Name(id='self', ctx=Load()),
                  attr='instance_var',
                  ctx=Store())],
              value=Constant(value=20))]),
        FunctionDef(
          name='add',
          args=arguments(
            args=[
              arg(arg='self'),
              arg(
                arg='a',
                annotation=Name(id='int', ctx=Load())),
              arg(
                ar

In [3]:
class ClassAnalyzer(ast.NodeVisitor):
    def __init__(self):
        self.class_stack = []  # Track nested class context
        self.methods_found = []

    def visit_ClassDef(self, node):
        # Build qualified class name
        self.class_stack.append(node.name)
        qualified_name = ".".join(self.class_stack)

        print(f"Class: {qualified_name}")
        print(f"  Base classes: {[base.id if isinstance(base, ast.Name) else '?' for base in node.bases]}")
        print(f"  Decorators: {len(node.decorator_list)}")

        # Analyze class body
        for item in node.body:
            if isinstance(item, ast.FunctionDef):
                method_qualified = f"{qualified_name}.{item.name}"
                self.methods_found.append(method_qualified)
                print(f"  Method: {item.name}")

                # Check if it's a special method
                if item.name.startswith("__") and item.name.endswith("__"):
                    print("    (magic method)")

                # Check for decorators (staticmethod, classmethod, etc.)
                for decorator in item.decorator_list:
                    if isinstance(decorator, ast.Name):
                        print(f"    Decorator: @{decorator.id}")

            elif isinstance(item, ast.Assign):
                # Class variable
                for target in item.targets:
                    if isinstance(target, ast.Name):
                        print(f"  Class variable: {target.id}")

            elif isinstance(item, ast.AnnAssign):
                # Annotated class variable
                if isinstance(item.target, ast.Name):
                    print(f"  Annotated class variable: {item.target.id}")

        # Visit nested items (including nested classes)
        self.generic_visit(node)

        # Pop class from stack when done
        self.class_stack.pop()


analyzer = ClassAnalyzer()
analyzer.visit(tree)
print(f"\nAll methods found: {analyzer.methods_found}")

Class: Calculator
  Base classes: ['BaseCalculator', 'Protocol']
  Decorators: 1
  Class variable: class_var
  Method: __init__
    (magic method)
  Method: add
  Method: static_method
    Decorator: @staticmethod
Class: Calculator.NestedClass
  Base classes: []
  Decorators: 0
  Method: nested_method

All methods found: ['Calculator.__init__', 'Calculator.add', 'Calculator.static_method', 'Calculator.NestedClass.nested_method']


The class stack pattern is crucial for handling nested classes correctly. Without it, we couldn't distinguish between `Outer.method` and `Outer.Inner.method`.

## 6. Call Nodes (Critical for Method Call Analysis)

Call nodes represent function and method calls. They're the heart of call counting functionality in static analysis tools. The structure of a Call node varies significantly depending on what's being called - a simple function, a method on an object, a class method, or something more complex. Understanding these patterns is essential for analyzing method call patterns and tracking function usage.

**ast.Call**

A Call node has three main attributes: `func` (what's being called), `args` (positional arguments), and `keywords` (keyword arguments). The `func` attribute is where the complexity lies - it can be a Name (simple function), an Attribute (method call), or even another Call (for chained calls).

In [4]:
code = """
# Different call patterns
result1 = print("hello")                    # Simple function call
result2 = calc.add(5, 10)                   # Instance method call
result3 = Calculator.static_method()        # Class/static method call
result4 = self.process()                    # Self method call
result5 = super().parent_method()           # Super call
result6 = get_calc().add(1, 2)              # Chained call
result7 = module.submodule.function()       # Qualified call
result8 = func(x=10, y=20)                  # Keyword arguments
"""

tree = ast.parse(code)
print(ast.dump(tree, indent=2))

Module(
  body=[
    Assign(
      targets=[
        Name(id='result1', ctx=Store())],
      value=Call(
        func=Name(id='print', ctx=Load()),
        args=[
          Constant(value='hello')])),
    Assign(
      targets=[
        Name(id='result2', ctx=Store())],
      value=Call(
        func=Attribute(
          value=Name(id='calc', ctx=Load()),
          attr='add',
          ctx=Load()),
        args=[
          Constant(value=5),
          Constant(value=10)])),
    Assign(
      targets=[
        Name(id='result3', ctx=Store())],
      value=Call(
        func=Attribute(
          value=Name(id='Calculator', ctx=Load()),
          attr='static_method',
          ctx=Load()))),
    Assign(
      targets=[
        Name(id='result4', ctx=Store())],
      value=Call(
        func=Attribute(
          value=Name(id='self', ctx=Load()),
          attr='process',
          ctx=Load()))),
    Assign(
      targets=[
        Name(id='result5', ctx=Store())],
      value=Call(
    

In [5]:
class CallPatternAnalyzer(ast.NodeVisitor):
    def visit_Call(self, node):
        print("Found call:")

        # Analyze what's being called
        if isinstance(node.func, ast.Name):
            # Simple function call: func()
            print(f"  Simple function: {node.func.id}()")

        elif isinstance(node.func, ast.Attribute):
            # Method/attribute call: something.method()
            attr_name = node.func.attr

            if isinstance(node.func.value, ast.Name):
                obj_name = node.func.value.id
                print(f"  Attribute call: {obj_name}.{attr_name}()")

                # Special cases we care about
                if obj_name == "self":
                    print("    -> This is a self method call")
                elif obj_name[0].isupper():
                    print("    -> Might be a class/static method call")
                else:
                    print("    -> Instance method call (NEED TO RESOLVE TYPE!)")

            elif isinstance(node.func.value, ast.Call):
                # Chained call: something().method()
                print(f"  Chained call: [expression].{attr_name}()")

            elif isinstance(node.func.value, ast.Attribute):
                # Nested attribute: module.submodule.func()
                print(f"  Nested attribute call ending in .{attr_name}()")

        elif isinstance(node.func, ast.Subscript):
            # Subscript call: functions_list[0]()
            print("  Subscript call: [expression][...]()")

        # Analyze arguments
        print(f"  Positional args: {len(node.args)}")
        print(f"  Keyword args: {len(node.keywords)}")

        # For debugging, show the full structure
        print(f"  Full structure: {ast.dump(node.func)[:100]}...")

        self.generic_visit(node)


analyzer = CallPatternAnalyzer()
analyzer.visit(tree)

Found call:
  Simple function: print()
  Positional args: 1
  Keyword args: 0
  Full structure: Name(id='print', ctx=Load())...
Found call:
  Attribute call: calc.add()
    -> Instance method call (NEED TO RESOLVE TYPE!)
  Positional args: 2
  Keyword args: 0
  Full structure: Attribute(value=Name(id='calc', ctx=Load()), attr='add', ctx=Load())...
Found call:
  Attribute call: Calculator.static_method()
    -> Might be a class/static method call
  Positional args: 0
  Keyword args: 0
  Full structure: Attribute(value=Name(id='Calculator', ctx=Load()), attr='static_method', ctx=Load())...
Found call:
  Attribute call: self.process()
    -> This is a self method call
  Positional args: 0
  Keyword args: 0
  Full structure: Attribute(value=Name(id='self', ctx=Load()), attr='process', ctx=Load())...
Found call:
  Chained call: [expression].parent_method()
  Positional args: 0
  Keyword args: 0
  Full structure: Attribute(value=Call(func=Name(id='super', ctx=Load())), attr='parent_method', 

The challenge in static analysis: When you see `calc.add()`, you need to know that `calc` is a `Calculator` instance. This requires tracking variable assignments to map `calc` to `Calculator`, which is the core of scope-aware variable tracking solutions.

## 7. Assignment Nodes (Key for Variable Tracking)

Assignment nodes are where we detect variable types for scope-aware tracking in static analysis. Python has multiple assignment node types, each with different structures. The key insight for variable tracking is that when we see `calc = Calculator()`, we can infer that `calc` is of type `Calculator`. Similarly, with type annotations like `calc: Calculator = get_calc()`, we can trust the annotation even if we can't analyze `get_calc()`.

**ast.Assign** (regular assignment)

Regular assignments can have multiple targets (for unpacking), but we typically only track simple single-target assignments where we can confidently determine the type. The `value` attribute contains the expression being assigned, which might be a constructor call, a function call, or any other expression.

In [6]:
code = """
# Simple assignments
calc = Calculator()                         # Constructor call
result = calc.add(5, 10)                    # Method call result
data = [1, 2, 3]                            # List literal

# Multiple targets (same value to multiple variables)
x = y = z = 10                              # Multiple assignment

# Tuple unpacking
a, b = 1, 2                                 # Tuple assignment
first, *rest = [1, 2, 3, 4]                 # Extended unpacking
"""

tree = ast.parse(code)
print(ast.dump(tree, indent=2))

Module(
  body=[
    Assign(
      targets=[
        Name(id='calc', ctx=Store())],
      value=Call(
        func=Name(id='Calculator', ctx=Load()))),
    Assign(
      targets=[
        Name(id='result', ctx=Store())],
      value=Call(
        func=Attribute(
          value=Name(id='calc', ctx=Load()),
          attr='add',
          ctx=Load()),
        args=[
          Constant(value=5),
          Constant(value=10)])),
    Assign(
      targets=[
        Name(id='data', ctx=Store())],
      value=List(
        elts=[
          Constant(value=1),
          Constant(value=2),
          Constant(value=3)],
        ctx=Load())),
    Assign(
      targets=[
        Name(id='x', ctx=Store()),
        Name(id='y', ctx=Store()),
        Name(id='z', ctx=Store())],
      value=Constant(value=10)),
    Assign(
      targets=[
        Tuple(
          elts=[
            Name(id='a', ctx=Store()),
            Name(id='b', ctx=Store())],
          ctx=Store())],
      value=Tuple(
        el

In [7]:
class AssignmentTracker(ast.NodeVisitor):
    def visit_Assign(self, node):
        print("Assignment found:")

        # Check targets (can be multiple)
        for target in node.targets:
            if isinstance(target, ast.Name):
                # Simple variable assignment
                print(f"  Target: {target.id}")

            elif isinstance(target, ast.Tuple) or isinstance(target, ast.List):
                # Tuple/list unpacking
                names = [t.id for t in target.elts if isinstance(t, ast.Name)]
                print(f"  Unpacking to: {names}")

            elif isinstance(target, ast.Starred):
                # Extended unpacking with *
                if isinstance(target.value, ast.Name):
                    print(f"  Starred target: *{target.value.id}")

        # Analyze the value being assigned
        if isinstance(node.value, ast.Call):
            if isinstance(node.value.func, ast.Name):
                func_name = node.value.func.id
                print(f"  Value: {func_name}() call")

                # Check if it looks like a constructor (capitalized)
                if func_name[0].isupper():
                    print(f"    -> Likely constructor for class {func_name}")

        elif isinstance(node.value, ast.Name):
            print(f"  Value: variable {node.value.id}")

        elif isinstance(node.value, ast.Constant):
            print(f"  Value: constant {node.value.value}")

        self.generic_visit(node)


tracker = AssignmentTracker()
tracker.visit(tree)

Assignment found:
  Target: calc
  Value: Calculator() call
    -> Likely constructor for class Calculator
Assignment found:
  Target: result
Assignment found:
  Target: data
Assignment found:
  Target: x
  Target: y
  Target: z
  Value: constant 10
Assignment found:
  Unpacking to: ['a', 'b']
Assignment found:
  Unpacking to: ['first']


**ast.AnnAssign** (annotated assignment)

Annotated assignments explicitly declare the variable's type, making them extremely valuable for type tracking. The annotation provides type information we can trust, even when we can't analyze the assigned value. This is particularly useful for function return values or complex expressions.

In [8]:
code = """
# Annotated assignments
calc: Calculator = Calculator()             # Type matches constructor
processor: DataProcessor = get_processor()  # Trust annotation over value
count: int = 0                              # Primitive type annotation
items: List[str] = []                       # Generic type annotation
maybe_calc: Optional[Calculator] = None     # Optional type

# Annotation without initial value
future_value: str                           # Declaration only, no assignment
"""

tree = ast.parse(code)
print(ast.dump(tree, indent=2))

Module(
  body=[
    AnnAssign(
      target=Name(id='calc', ctx=Store()),
      annotation=Name(id='Calculator', ctx=Load()),
      value=Call(
        func=Name(id='Calculator', ctx=Load())),
      simple=1),
    AnnAssign(
      target=Name(id='processor', ctx=Store()),
      annotation=Name(id='DataProcessor', ctx=Load()),
      value=Call(
        func=Name(id='get_processor', ctx=Load())),
      simple=1),
    AnnAssign(
      target=Name(id='count', ctx=Store()),
      annotation=Name(id='int', ctx=Load()),
      value=Constant(value=0),
      simple=1),
    AnnAssign(
      target=Name(id='items', ctx=Store()),
      annotation=Subscript(
        value=Name(id='List', ctx=Load()),
        slice=Name(id='str', ctx=Load()),
        ctx=Load()),
      value=List(ctx=Load()),
      simple=1),
    AnnAssign(
      target=Name(id='maybe_calc', ctx=Store()),
      annotation=Subscript(
        value=Name(id='Optional', ctx=Load()),
        slice=Name(id='Calculator', ctx=Load()),
    

In [9]:
class AnnotatedAssignmentTracker(ast.NodeVisitor):
    def visit_AnnAssign(self, node):
        print("Annotated assignment:")

        # Target is always a single name (unlike regular Assign)
        if isinstance(node.target, ast.Name):
            print(f"  Variable: {node.target.id}")

        # Analyze the annotation
        if isinstance(node.annotation, ast.Name):
            # Simple type: int, str, Calculator
            print(f"  Type annotation: {node.annotation.id}")

        elif isinstance(node.annotation, ast.Constant):
            # String annotation: "Calculator" (forward reference)
            if isinstance(node.annotation.value, str):
                print(f"  String annotation: '{node.annotation.value}'")

        elif isinstance(node.annotation, ast.Subscript):
            # Generic type: List[int], Optional[Calculator]
            if isinstance(node.annotation.value, ast.Name):
                print(f"  Generic type: {node.annotation.value.id}[...]")

        # Check if there's an actual assignment (value can be None)
        if node.value is not None:
            print("  Has initial value: Yes")
            if isinstance(node.value, ast.Call) and isinstance(node.value.func, ast.Name):
                print(f"    Value: {node.value.func.id}() call")
        else:
            print("  Has initial value: No (declaration only)")

        # The simple field indicates whether the assignment target is simple
        # x: int = 5 would have simple=1, (x): int = 5 would have simple=0
        print(f"  Simple: {node.simple}")

        self.generic_visit(node)


tracker = AnnotatedAssignmentTracker()
tracker.visit(tree)

Annotated assignment:
  Variable: calc
  Type annotation: Calculator
  Has initial value: Yes
    Value: Calculator() call
  Simple: 1
Annotated assignment:
  Variable: processor
  Type annotation: DataProcessor
  Has initial value: Yes
    Value: get_processor() call
  Simple: 1
Annotated assignment:
  Variable: count
  Type annotation: int
  Has initial value: Yes
  Simple: 1
Annotated assignment:
  Variable: items
  Generic type: List[...]
  Has initial value: Yes
  Simple: 1
Annotated assignment:
  Variable: maybe_calc
  Generic type: Optional[...]
  Has initial value: Yes
  Simple: 1
Annotated assignment:
  Variable: future_value
  Type annotation: str
  Has initial value: No (declaration only)
  Simple: 1


For scope-aware variable tracking, we primarily focus on simple annotations (ast.Name) and string annotations (ast.Constant with string value) as these give us clear type names we can use for resolution.

## 8. Name and Attribute Nodes

Name and Attribute nodes are the building blocks of variable and method references. Every time you reference a variable or access an attribute, these nodes are created. Understanding their structure is essential for both tracking variable usage and resolving method calls.

**ast.Name** (simple variable reference)

A Name node represents a simple identifier - a variable, function name, or class name. The `id` attribute contains the actual name as a string, and the `ctx` attribute tells you whether the name is being read, written, or deleted. Name nodes are what we track when building our scope-aware variable dictionary.

In [10]:
code = """
# Various uses of names
Calculator = imported_class              # Both sides are Name nodes
x = 5                                    # x is Name with Store context
y = x + 10                               # x is Name with Load context
print(y)                                 # print and y are Names with Load
del x                                    # x is Name with Del context

def process(data):                      # process and data are names
    return len(data)                    # len and data are Names with Load
"""

tree = ast.parse(code)
print(ast.dump(tree, indent=2))

Module(
  body=[
    Assign(
      targets=[
        Name(id='Calculator', ctx=Store())],
      value=Name(id='imported_class', ctx=Load())),
    Assign(
      targets=[
        Name(id='x', ctx=Store())],
      value=Constant(value=5)),
    Assign(
      targets=[
        Name(id='y', ctx=Store())],
      value=BinOp(
        left=Name(id='x', ctx=Load()),
        op=Add(),
        right=Constant(value=10))),
    Expr(
      value=Call(
        func=Name(id='print', ctx=Load()),
        args=[
          Name(id='y', ctx=Load())])),
    Delete(
      targets=[
        Name(id='x', ctx=Del())]),
    FunctionDef(
      name='process',
      args=arguments(
        args=[
          arg(arg='data')]),
      body=[
        Return(
          value=Call(
            func=Name(id='len', ctx=Load()),
            args=[
              Name(id='data', ctx=Load())]))])])


In [11]:
class NameAnalyzer(ast.NodeVisitor):
    def __init__(self):
        self.names_by_context = {"Load": [], "Store": [], "Del": []}

    def visit_Name(self, node):
        context = type(node.ctx).__name__
        self.names_by_context[context].append(node.id)
        print(f"Name: '{node.id}' (context: {context}, line: {node.lineno})")

        # Names can appear anywhere - in expressions, statements, etc.
        # Common patterns:
        if node.id == "self":
            print("  -> Found 'self' reference")
        elif node.id[0].isupper():
            print("  -> Possibly a class name")
        elif node.id.startswith("__") and node.id.endswith("__"):
            print("  -> Magic name/built-in")

        self.generic_visit(node)


analyzer = NameAnalyzer()
analyzer.visit(tree)
print("\nSummary:")
print(f"  Variables read: {analyzer.names_by_context['Load']}")
print(f"  Variables written: {analyzer.names_by_context['Store']}")
print(f"  Variables deleted: {analyzer.names_by_context['Del']}")

Name: 'Calculator' (context: Store, line: 3)
  -> Possibly a class name
Name: 'imported_class' (context: Load, line: 3)
Name: 'x' (context: Store, line: 4)
Name: 'y' (context: Store, line: 5)
Name: 'x' (context: Load, line: 5)
Name: 'print' (context: Load, line: 6)
Name: 'y' (context: Load, line: 6)
Name: 'x' (context: Del, line: 7)
Name: 'len' (context: Load, line: 10)
Name: 'data' (context: Load, line: 10)

Summary:
  Variables read: ['imported_class', 'x', 'print', 'y', 'len', 'data']
  Variables written: ['Calculator', 'x', 'y']
  Variables deleted: ['x']


**ast.Attribute** (attribute access)

An Attribute node represents accessing an attribute of an object (using the dot operator). The `value` attribute is the object being accessed (often a Name node), and the `attr` attribute is a string containing the attribute name. This is crucial for method calls like `calc.add()` where the Call node's func is an Attribute.

In [12]:
code = """
# Attribute access patterns
result = obj.attribute                  # Simple attribute access
value = self.instance_var               # Instance variable
self.method()                            # Method call via self
Calculator.static_method()              # Class attribute/method
module.submodule.function()             # Nested attributes
(a + b).bit_length()                    # Attribute of expression
"""

tree = ast.parse(code)
print(ast.dump(tree, indent=2))

Module(
  body=[
    Assign(
      targets=[
        Name(id='result', ctx=Store())],
      value=Attribute(
        value=Name(id='obj', ctx=Load()),
        attr='attribute',
        ctx=Load())),
    Assign(
      targets=[
        Name(id='value', ctx=Store())],
      value=Attribute(
        value=Name(id='self', ctx=Load()),
        attr='instance_var',
        ctx=Load())),
    Expr(
      value=Call(
        func=Attribute(
          value=Name(id='self', ctx=Load()),
          attr='method',
          ctx=Load()))),
    Expr(
      value=Call(
        func=Attribute(
          value=Name(id='Calculator', ctx=Load()),
          attr='static_method',
          ctx=Load()))),
    Expr(
      value=Call(
        func=Attribute(
          value=Attribute(
            value=Name(id='module', ctx=Load()),
            attr='submodule',
            ctx=Load()),
          attr='function',
          ctx=Load()))),
    Expr(
      value=Call(
        func=Attribute(
          value=BinOp(

In [13]:
class AttributeAnalyzer(ast.NodeVisitor):
    def visit_Attribute(self, node):
        print(f"Attribute access: .{node.attr}")
        print(f"  Context: {type(node.ctx).__name__}")

        # Analyze what the attribute is accessed on
        if isinstance(node.value, ast.Name):
            base_name = node.value.id
            print(f"  Base object: {base_name}")

            # Pattern recognition
            if base_name == "self":
                print(f"    -> Instance attribute/method: self.{node.attr}")
            elif base_name[0].isupper():
                print(f"    -> Class attribute/method: {base_name}.{node.attr}")
            else:
                print(f"    -> Object attribute: {base_name}.{node.attr}")

        elif isinstance(node.value, ast.Attribute):
            # Nested attribute (like module.submodule.attr)
            print("  Base: nested attribute chain")

            # Build the full chain
            chain = [node.attr]
            current = node.value
            while isinstance(current, ast.Attribute):
                chain.append(current.attr)
                current = current.value
            if isinstance(current, ast.Name):
                chain.append(current.id)
            chain.reverse()
            print(f"    -> Full chain: {'.'.join(chain)}")

        elif isinstance(node.value, ast.Call):
            print("  Base: function call result")
            print(f"    -> Chained call: [call_result].{node.attr}")

        else:
            print(f"  Base: {type(node.value).__name__} expression")

        self.generic_visit(node)


analyzer = AttributeAnalyzer()
analyzer.visit(tree)

Attribute access: .attribute
  Context: Load
  Base object: obj
    -> Object attribute: obj.attribute
Attribute access: .instance_var
  Context: Load
  Base object: self
    -> Instance attribute/method: self.instance_var
Attribute access: .method
  Context: Load
  Base object: self
    -> Instance attribute/method: self.method
Attribute access: .static_method
  Context: Load
  Base object: Calculator
    -> Class attribute/method: Calculator.static_method
Attribute access: .function
  Context: Load
  Base: nested attribute chain
    -> Full chain: module.submodule.function
Attribute access: .submodule
  Context: Load
  Base object: module
    -> Object attribute: module.submodule
Attribute access: .bit_length
  Context: Load
  Base: BinOp expression


The interplay between Name and Attribute nodes is key to understanding Python's attribute access patterns. In `calc.add()`, `calc` is a Name node, and the entire `calc.add` is an Attribute node, which then becomes the func of a Call node.

## 9. Simple Annotations

Simple annotations are the most common and easiest to handle. They're just Name nodes representing the type directly. These are perfect for variable tracking because we get a clear, unambiguous type name. When we see `calc: Calculator`, we know exactly what type `calc` should be, making it reliable for resolving method calls later.

In [14]:
code = """
# Simple type annotations
def process(x: int, calc: Calculator, name: str) -> bool:
    result: float = calc.compute(x)
    status: bool = True
    return status

class DataHandler:
    count: int = 0
    processor: Calculator

    def handle(self, data: bytes) -> None:
        pass
"""

tree = ast.parse(code)
print(ast.dump(tree, indent=2))

Module(
  body=[
    FunctionDef(
      name='process',
      args=arguments(
        args=[
          arg(
            arg='x',
            annotation=Name(id='int', ctx=Load())),
          arg(
            arg='calc',
            annotation=Name(id='Calculator', ctx=Load())),
          arg(
            arg='name',
            annotation=Name(id='str', ctx=Load()))]),
      body=[
        AnnAssign(
          target=Name(id='result', ctx=Store()),
          annotation=Name(id='float', ctx=Load()),
          value=Call(
            func=Attribute(
              value=Name(id='calc', ctx=Load()),
              attr='compute',
              ctx=Load()),
            args=[
              Name(id='x', ctx=Load())]),
          simple=1),
        AnnAssign(
          target=Name(id='status', ctx=Store()),
          annotation=Name(id='bool', ctx=Load()),
          value=Constant(value=True),
          simple=1),
        Return(
          value=Name(id='status', ctx=Load()))],
      returns=Na

In [15]:
class SimpleAnnotationExtractor(ast.NodeVisitor):
    def __init__(self):
        self.annotations_found = []

    def visit_FunctionDef(self, node):
        print(f"Function: {node.name}")

        # Check parameter annotations
        for arg in node.args.args:
            if arg.annotation and isinstance(arg.annotation, ast.Name):
                type_name = arg.annotation.id
                self.annotations_found.append((arg.arg, type_name))
                print(f"  Parameter {arg.arg}: {type_name}")

                # Determine if it's a built-in or custom type
                if type_name in ["int", "str", "float", "bool", "bytes", "dict", "list", "tuple", "set"]:
                    print("    -> Built-in type")
                elif type_name[0].isupper():
                    print("    -> Likely a class type")

        # Check return annotation
        if node.returns and isinstance(node.returns, ast.Name):
            print(f"  Returns: {node.returns.id}")
            self.annotations_found.append(("return", node.returns.id))

        self.generic_visit(node)

    def visit_AnnAssign(self, node):
        # Variable annotations
        if isinstance(node.target, ast.Name) and isinstance(node.annotation, ast.Name):
            var_name = node.target.id
            type_name = node.annotation.id
            self.annotations_found.append((var_name, type_name))
            print(f"Variable {var_name}: {type_name}")

        self.generic_visit(node)


extractor = SimpleAnnotationExtractor()
extractor.visit(tree)
print(f"\nAll annotations: {extractor.annotations_found}")

Function: process
  Parameter x: int
    -> Built-in type
  Parameter calc: Calculator
    -> Likely a class type
  Parameter name: str
    -> Built-in type
  Returns: bool
Variable result: float
Variable status: bool
Variable count: int
Variable processor: Calculator
Function: handle
  Parameter data: bytes
    -> Built-in type

All annotations: [('x', 'int'), ('calc', 'Calculator'), ('name', 'str'), ('return', 'bool'), ('result', 'float'), ('status', 'bool'), ('count', 'int'), ('processor', 'Calculator'), ('data', 'bytes')]


## 10. String Annotations (Forward References)

String annotations are used for forward references - when you need to reference a class that hasn't been defined yet, or to avoid circular imports. Python treats string annotations as ast.Constant nodes with string values. These are important for variable tracking because they're commonly used in real codebases, especially with the `from __future__ import annotations` pattern.

In [16]:
code = """
from __future__ import annotations  # Makes ALL annotations strings

class Node:
    def set_parent(self, parent: "Node") -> None:  # Forward reference
        self.parent = parent

    def get_children(self) -> List["Node"]:  # Forward reference in generic
        return self.children

def process_handler(h: "Handler") -> "Result":  # Both are strings
    return h.handle()

# Circular dependency case
class A:
    def use_b(self, b: "B") -> None:
        pass

class B:
    def use_a(self, a: A) -> None:  # A is available, no string needed
        pass
"""

tree = ast.parse(code)
print(ast.dump(tree, indent=2))

Module(
  body=[
    ImportFrom(
      module='__future__',
      names=[
        alias(name='annotations')],
      level=0),
    ClassDef(
      name='Node',
      body=[
        FunctionDef(
          name='set_parent',
          args=arguments(
            args=[
              arg(arg='self'),
              arg(
                arg='parent',
                annotation=Constant(value='Node'))]),
          body=[
            Assign(
              targets=[
                Attribute(
                  value=Name(id='self', ctx=Load()),
                  attr='parent',
                  ctx=Store())],
              value=Name(id='parent', ctx=Load()))],
          returns=Constant(value=None)),
        FunctionDef(
          name='get_children',
          args=arguments(
            args=[
              arg(arg='self')]),
          body=[
            Return(
              value=Attribute(
                value=Name(id='self', ctx=Load()),
                attr='children',
                

In [17]:
class StringAnnotationAnalyzer(ast.NodeVisitor):
    def analyze_annotation(self, annotation, context=""):
        """Recursively analyze an annotation node."""
        if isinstance(annotation, ast.Constant):
            if isinstance(annotation.value, str):
                print(f"{context}String annotation: '{annotation.value}'")

                # Try to identify what kind of type it represents
                value = annotation.value
                if value in ["int", "str", "float", "bool"]:
                    print(f"{context}  -> Built-in type as string")
                elif value[0].isupper() if value else False:
                    print(f"{context}  -> Class name as string")
                elif "[" in value:
                    print(f"{context}  -> Complex generic type as string")

                return annotation.value

        elif isinstance(annotation, ast.Name):
            print(f"{context}Direct annotation: {annotation.id}")
            return annotation.id

        elif isinstance(annotation, ast.Subscript):
            print(f"{context}Generic type annotation")
            # Handle List["Node"] pattern
            if isinstance(annotation.slice, ast.Constant):
                self.analyze_annotation(annotation.slice, context + "  ")

        return None

    def visit_FunctionDef(self, node):
        print(f"\nFunction: {node.name}")

        # Check parameters
        for arg in node.args.args:
            if arg.annotation:
                print(f"  Parameter '{arg.arg}':")
                self.analyze_annotation(arg.annotation, "    ")

        # Check return
        if node.returns:
            print("  Return type:")
            self.analyze_annotation(node.returns, "    ")

        self.generic_visit(node)


analyzer = StringAnnotationAnalyzer()
analyzer.visit(tree)


Function: set_parent
  Parameter 'parent':
    String annotation: 'Node'
      -> Class name as string
  Return type:

Function: get_children
  Return type:
    Generic type annotation
      String annotation: 'Node'
        -> Class name as string

Function: process_handler
  Parameter 'h':
    String annotation: 'Handler'
      -> Class name as string
  Return type:
    String annotation: 'Result'
      -> Class name as string

Function: use_b
  Parameter 'b':
    String annotation: 'B'
      -> Class name as string
  Return type:

Function: use_a
  Parameter 'a':
    Direct annotation: A
  Return type:


## 11. Complex Annotations (Not Handled in Basic Implementations)

Complex annotations include generics, unions, optionals, and other advanced type hints. While basic static analysis implementations don't handle these, understanding their AST structure is important for future enhancements and for knowing what we're explicitly choosing not to support.

In [18]:
code = """
from typing import Optional, Union, List, Dict, Callable, TypeVar

T = TypeVar('T')

def complex_signatures(
    # Union types (Python 3.10+ can use | operator)
    value: Union[int, str],
    modern_union: int | str,

    # Optional (equivalent to Union[T, None])
    maybe_calc: Optional[Calculator],

    # Generics with type parameters
    items: List[str],
    mapping: Dict[str, int],
    nested: List[Dict[str, Calculator]],

    # Callable signatures
    callback: Callable[[int, str], bool],

    # Type variables
    generic: T,
) -> Optional[List[T]]:
    pass
"""

tree = ast.parse(code)
print(ast.dump(tree, indent=2))

Module(
  body=[
    ImportFrom(
      module='typing',
      names=[
        alias(name='Optional'),
        alias(name='Union'),
        alias(name='List'),
        alias(name='Dict'),
        alias(name='Callable'),
        alias(name='TypeVar')],
      level=0),
    Assign(
      targets=[
        Name(id='T', ctx=Store())],
      value=Call(
        func=Name(id='TypeVar', ctx=Load()),
        args=[
          Constant(value='T')])),
    FunctionDef(
      name='complex_signatures',
      args=arguments(
        args=[
          arg(
            arg='value',
            annotation=Subscript(
              value=Name(id='Union', ctx=Load()),
              slice=Tuple(
                elts=[
                  Name(id='int', ctx=Load()),
                  Name(id='str', ctx=Load())],
                ctx=Load()),
              ctx=Load())),
          arg(
            arg='modern_union',
            annotation=BinOp(
              left=Name(id='int', ctx=Load()),
              op=BitOr

In [19]:
class ComplexAnnotationAnalyzer(ast.NodeVisitor):
    def analyze_complex_annotation(self, node, depth=0):
        indent = "  " * depth

        if isinstance(node, ast.Name):
            print(f"{indent}Simple: {node.id}")

        elif isinstance(node, ast.Constant):
            print(f"{indent}String: '{node.value}'")

        elif isinstance(node, ast.Subscript):
            # Generic type like List[int], Optional[Calculator]
            if isinstance(node.value, ast.Name):
                print(f"{indent}Generic: {node.value.id}[...]")

                # Analyze the type parameter(s)
                if isinstance(node.slice, ast.Name):
                    print(f"{indent}  Type param: {node.slice.id}")
                elif isinstance(node.slice, ast.Tuple):
                    # Multiple type params like Dict[str, int]
                    print(f"{indent}  Type params:")
                    for elt in node.slice.elts:
                        self.analyze_complex_annotation(elt, depth + 2)
                else:
                    self.analyze_complex_annotation(node.slice, depth + 1)

        elif isinstance(node, ast.BinOp):
            # Union type using | operator (Python 3.10+)
            if isinstance(node.op, ast.BitOr):
                print(f"{indent}Union (| operator):")
                self.analyze_complex_annotation(node.left, depth + 1)
                print(f"{indent}  or")
                self.analyze_complex_annotation(node.right, depth + 1)

        elif isinstance(node, ast.Attribute):
            # Qualified type like typing.Optional
            chain = []
            current = node
            while isinstance(current, ast.Attribute):
                chain.append(current.attr)
                current = current.value
            if isinstance(current, ast.Name):
                chain.append(current.id)
            chain.reverse()
            print(f"{indent}Qualified: {'.'.join(chain)}")

        else:
            print(f"{indent}Unknown annotation type: {type(node).__name__}")

    def visit_FunctionDef(self, node):
        print(f"\nFunction: {node.name}")

        for arg in node.args.args:
            if arg.annotation:
                print(f"  Parameter '{arg.arg}':")
                self.analyze_complex_annotation(arg.annotation, 2)

        if node.returns:
            print("  Returns:")
            self.analyze_complex_annotation(node.returns, 2)

        self.generic_visit(node)


analyzer = ComplexAnnotationAnalyzer()
analyzer.visit(tree)


Function: complex_signatures
  Parameter 'value':
    Generic: Union[...]
      Type params:
        Simple: int
        Simple: str
  Parameter 'modern_union':
    Union (| operator):
      Simple: int
      or
      Simple: str
  Parameter 'maybe_calc':
    Generic: Optional[...]
      Type param: Calculator
  Parameter 'items':
    Generic: List[...]
      Type param: str
  Parameter 'mapping':
    Generic: Dict[...]
      Type params:
        Simple: str
        Simple: int
  Parameter 'nested':
    Generic: List[...]
      Generic: Dict[...]
        Type params:
          Simple: str
          Simple: Calculator
  Parameter 'callback':
    Generic: Callable[...]
      Type params:
        Unknown annotation type: List
        Simple: bool
  Parameter 'generic':
    Simple: T
  Returns:
    Generic: Optional[...]
      Generic: List[...]
        Type param: T


## Summary

Understanding these complex patterns helps us make informed decisions about what to support. For basic static analysis implementations, we can consciously choose to support only simple Name and string Constant annotations, as they cover the majority of use cases while keeping the implementation manageable.

### Key Takeaways:

1. **Simple Annotations**: Direct `ast.Name` nodes - easiest to extract and use
2. **String Annotations**: `ast.Constant` nodes with string values - common for forward references
3. **Complex Annotations**: Various combinations of `ast.Subscript`, `ast.BinOp`, etc. - powerful but complex to handle

The annotation tracking system focuses on the first two categories as they provide the most value with the least complexity.

## Next Steps

You've now covered the core AST node types essential for understanding class structures, method calls, assignments, and type annotations. 

**Continue to Part 3**: [AST Visitor Patterns](./3-ast-visitor-patterns.ipynb) to learn about advanced visitor patterns, context stacks, error handling, and practical debugging techniques that will help you implement robust AST analysis for static analysis projects.

Part 3 will cover:
- Maintaining context stacks for scope tracking
- Building qualified names correctly
- Implementing scope-aware variable tracking
- Using `ast.dump()` for debugging
- Error handling best practices
- A complete working example that demonstrates these concepts